In [1]:
import netsquid as ns
import numpy as np
import cmath
import random
import netsquid.components.instructions as instr
from netsquid.components.qprocessor import QuantumProcessor
from netsquid.components.qprocessor import PhysicalInstruction
from netsquid.nodes.connections import Connection, DirectConnection
from netsquid.components import ClassicalChannel
from netsquid.components.models import FibreDelayModel
from netsquid.nodes import Node
from netsquid.components import QuantumChannel
from netsquid.qubits.qubitapi import create_qubits
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel,T1T2NoiseModel
from netsquid.protocols import NodeProtocol, Signals ,LocalProtocol  

In [2]:

class ClassicalBiConnection(DirectConnection):
    def __init__(self, length,name="ClassicalConnection"):
        
        super().__init__(name=name)
        self.add_subcomponent(ClassicalChannel("Channel_A2B", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("A", "send")],
                              forward_output=[("B", "recv")])
        self.add_subcomponent(ClassicalChannel("Channel_B2A", length=length,
                                               models={"delay_model": FibreDelayModel()}),
                              forward_input=[("B", "send")],
                              forward_output=[("A", "recv")])


class QuantumConnection(Connection):
    def __init__(self, length, prob,name="QuantumConnection"):
        super().__init__(name=name)
        self.prob = prob
        Model = DepolarNoiseModel(depolar_rate = self.prob,time_independent=True)
#         Model = DephaseNoiseModel(dephase_rate  = self.prob,time_independent=True)
        qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
                                      models={"delay_model": FibreDelayModel(), "quantum_noise_model" : Model})
        # Add channels and forward quantum channel output to external port output:
        self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        
# class QuantumConnection(Connection):
#     def __init__(self, length, name="QuantumConnection"):
#         super().__init__(name=name)
#         qchannel_a2b = QuantumChannel("qchannel_a2b", length=length,
#                                       models={"delay_model": FibreDelayModel(), "quantum_noise_model" : T1T2NoiseModel(T1 = 10)})
#         # Add channels and forward quantum channel output to external port output:
#         self.add_subcomponent(qchannel_a2b,forward_input=[("A","send")],forward_output=[("B", "recv")])
        #Connect qsource output to quantum channel input:
#         qsource.ports["qout0"].connect(qchannel_c2a.ports["send"])
#         qsource.ports["qout1"].connect(qchannel_c2b.ports["send"])


In [3]:
def create_processor(prob):
    def RandUnitary(prob):
        basis_matrix = np.identity(2)
        R= np.zeros(2)
        Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-prob)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    R = RandUnitary(prob)
    R1 =  ns.qubits.operators.Operator("R1", R)
    INSTR_R = instr.IGate("R_gate", R1)
    INSTR_I = instr.IInit()
    # We'll give both Alice and Bob the same kind of processor
    num_qubits = 4
    physical_instructions = [
        PhysicalInstruction(INSTR_I, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(INSTR_R, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False)
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,phys_instructions=physical_instructions)
    return processor



def create_processor1(probs):

    def RandUnitary():
        basis_matrix = np.identity(2)
        R= np.zeros(2)
        Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-Theta)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    R = RandUnitary()
    R1 =  ns.qubits.operators.Operator("R1", R)
    INSTR_R = instr.IGate("R_gate", R1)
    INSTR_I = instr.IInit()
    # We'll give both Alice and Bob the same kind of processor
    num_qubits = 4
    physical_instructions = [
        PhysicalInstruction(INSTR_I, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True),
        PhysicalInstruction(INSTR_R, duration=1, parallel=True),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False)
#         PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
#     memory_noise_model = DephaseNoiseModel(dephase_rate  = probs,time_independent=True)
    memory_noise_model = DepolarNoiseModel(depolar_rate  = probs,time_independent=True)
    processor = QuantumProcessor("quantum_processor", num_positions=num_qubits,mem_noise_models=memory_noise_model,phys_instructions=physical_instructions)
    return processor

In [4]:
from netsquid.components.qprogram import QuantumProgram

class InitStateProgram(QuantumProgram):
    default_num_qubits = 4   
    def program(self):
#         self.num_qubits = int(np.log2(self.num_qubits))
        def init_state():
            basis_matrix = np.identity(16)
            state = np.zeros((16,1))
            for i in range(16):
                if (i == 3) or (i== 12):
                    state = state + (1/np.sqrt(3))*basis_matrix[:,i].reshape((16,1))
                elif i == 5 or i==6 or i==9 or i==10 or i==12 :
                    state = state - (1/(2*np.sqrt(3)))*basis_matrix[:,i].reshape((16,1))
            return state
        qstate = init_state()
        q1,q2,q3,q4 = ns.qubits.create_qubits(4)
        ns.qubits.assign_qstate([q1,q2,q3,q4],qstate)
        qa,qb,qc,qd = self.get_qubit_indices()
        INSTR_I = instr.IInit()
        self.apply(INSTR_I,[qa,qb,qc,qd],qubits = [q1,q2,q3,q4])
        yield self.run()
        
class RandUnitary(QuantumProgram):
    def RandUnitary(self,prob):
        basis_matrix = np.identity(2)
        R= np.zeros(2)
#         Theta = np.random.uniform(0,2*np.pi)
        z = cmath.exp((-prob)*1j)
        R = R + basis_matrix[:,0].reshape((2,1))*np.transpose(basis_matrix[:,0].reshape((2,1))) + z*(basis_matrix[:,1].reshape((2,1))*np.transpose(basis_matrix[:,1].reshape((2,1))))
        return R
    
    def program(self,prob):
        R = self.RandUnitary(prob)
        R1 =  ns.qubits.operators.Operator("R1", R)
        INSTR_R = instr.IGate("R_gate", R1)
        self.apply(INSTR_R, 0)
        yield self.run()
        
class MeasureZ(QuantumProgram):
#     default_num_qubits = 4
    def program(self,mem_pos):
        qubits = self.get_qubit_indices()
        for i in range(len(mem_pos)):
            self.apply(instr.INSTR_MEASURE,qubits[mem_pos[i]], output_key="M"+str(mem_pos[i]))
        yield self.run()
        
class MeasureX(QuantumProgram):
    def program(self,mem_pos):
        qubits = self.get_qubit_indices()
        for i in range(len(mem_pos)):
            self.apply(instr.INSTR_H, qubits[mem_pos[i]])
            self.apply(instr.INSTR_MEASURE,qubits[mem_pos[i]], output_key="M"+str(mem_pos[i]))
        yield self.run()

In [5]:
class SourceInit(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
       
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase
       
        qubit_number = 4
        #Program to initialize the qubits in the memory, input param: number of qubits
        qubit_init_program = InitStateProgram(num_qubits=qubit_number)
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])
#         print(list_classic)
#         print(list_quantum)
        
#         print(self.node.name[1])
        node_num = int(self.node.name.replace('P','')) # Current Node Number    
        
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Init qubits in memory
#             print("Loop start")
#             self.node.qmemory.peek([0,1,2,3])
            self.node.qmemory.execute_program(qubit_init_program)
            
#             print(f"Node {node_num} init qubit program")
#             yield self.await_program(self.node.qmemory)
            expr = yield (self.await_program(self.node.qmemory))
#             print(self.node.qmemory.measure())
            #Send 1 qubit to first party
            qubit1 = self.node.qmemory.pop(positions=[0,1])
            self.node.ports[list_quantum[0]].tx_output(qubit1) 
#             print(f"Node {node_num} send qubit to Node {list_quantum[0][-1]}")
            qubit2 = self.node.qmemory.pop(positions=2)
            self.node.ports[list_quantum[1]].tx_output(qubit2)
#             print(f"Node {node_num} send qubit to Node {list_quantum[1][-1]}")
            qubit3 = self.node.qmemory.pop(positions=3)
            self.node.ports[list_quantum[2]].tx_output(qubit3)
#             print(f"Node {node_num} send qubit to Node {list_quantum[2][-1]}")
#             i=0
#             while (i<self.num_nodes-1):
#                 yield self.await_port_input(self.node.ports[list_classic[-1-i]])
#                 print(f"Node {node_num} wait ACK from Node {list_quantum[-1-i][-1]}")
#                 message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                 print(message)
#                 i = i+1
            
            
#             print(f"Node {node_num} send qubit to Node {list_quantum[1][-1]}")
            
#           Wait for ACK
            i=0
            while (i<self.num_nodes-1):
                if len(self.node.ports[list_classic[-1-i]].input_queue) != 0:
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
#                     print(f"Queue case from node {list_classic[-1-i]}")
                    message = self.node.ports[list_classic[-1-i]].input_queue[0][1].items
                    self.node.ports[list_classic[-1-i]].input_queue[0][1].items = []
#                     print(self.node.ports[list_classic[-1-i]].input_queue[0][1].items)
                else:
#                     print(f"Node 1 waitting from node {list_classic[-1-i][-1]}")
#                     print(f"Node {node_num} wait ACK from Node {list_quantum[-1-i][-1]}")
                    yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                    message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                     print(message)
                i = i+1
                
            #Measure qubit
class RecvMeas_A(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 2
        #Program to initialize the qubits in the memory, input param: number of qubits
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)
        
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])

        node_num = int(self.node.name.replace('P','')) # Current Node Number    
#         print(list_classic)
#         print(list_quantum)
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Wait for qubit
            yield self.await_port_input(self.node.ports[list_quantum[0]])
            
            #Measure qubit
            c = random.randint(0,1)
#             c = 0
#             print(c)
            if c == 0:
#                 print(f"Node {node_num} measure in Z basis ")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[0,1])
#                 print(f"Node {node_num} output")
#                 print(measure_program1.output)
#                 self.node.qmemory.discard(0)
            else:
#                 print(f"Node {node_num} measure in X basis ")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[0,1])
#                 print(f"Node {node_num} output")
#                 print(measure_program2.output)
#                 self.node.qmemory.discard(0)
            basis_sum = c

            i=0
            while (i<self.num_nodes-2):
#                 print(f"Node 1 await from node {list_classic[-1-i][-1]}")
                yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                 print(message)
                basis_sum = basis_sum + message
                i = i+1
            
            #Send  basis
#             print(f"Node 1 send basis to Node {list_classic[1][-1]}")
            self.node.ports[list_classic[1]].tx_output(c)
#             print(f"Node 1 send basis to Node {list_classic[2][-1]}")
            self.node.ports[list_classic[2]].tx_output(c)
            
            if (basis_sum % (self.num_nodes-1) == 0):
#                 print(f"List record index {x}")
                if c == 0:
                    if (measure_program1.output["M0"][0] == 0) and (measure_program1.output["M1"][0] == 0) :
                        global_list[x][0] = 1
                    elif (measure_program1.output["M0"][0] == 1) and (measure_program1.output["M1"][0] == 1) :
                        global_list[x][0] = 0
                    else:
                        global_list[x][0] = 2
                else:
                    if (measure_program2.output["M0"][0] == 0) and (measure_program2.output["M1"][0] == 0) :
                        global_list[x][0] = 1
                    elif (measure_program2.output["M0"][0] == 1) and (measure_program2.output["M1"][0] == 1) :
                        global_list[x][0] = 0
                    else:
                        global_list[x][0] = 2
                x = x+1
                basis_sum = 0
                if (x > self.list_length-1):
                    if node_num == 3:
#                         print(f"List distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
            self.node.ports[list_classic[0]].tx_output("ACK")
#             print("Node 1 send ACK")
#             #Send measurement results
#             yield self.await_port_input(self.node.ports[list_classic[0]])
#             message = self.node.ports[list_classic[0]].rx_input().items[0]
#             print(message)
            
class RecvMeas_B(NodeProtocol):
    def __init__(self, node ,name, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 1
        #Program to initialize the qubits in the memory, input param: number of qubits
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)
        randU_program = RandUnitary()
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])

        node_num = int(self.node.name.replace('P','')) # Current Node Number    
#         print(list_classic)
#         print(list_quantum)
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Wait for qubit
            yield self.await_port_input(self.node.ports[list_quantum[0]])
#             print(f"Node {node_num} received qubit")
#             print(self.node.qmemory.peek(0))
#             yield self.node.qmemory.execute_program(randU_program)
            #Measure qubit
            c = random.randint(0,1)
#             c = 0
#             print(c)
            if c == 0:
#                 print(f"Node {node_num} measure in Z basis ")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program1.output)
#                 self.node.qmemory.discard(0)
            else:
#                 print(f"Node {node_num} measure in X basis ")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program2.output)
#                 self.node.qmemory.discard(0)
            basis_sum = c

            i=0
            while (i<self.num_nodes-2):
#                 print(f"Node {node_num} Loop for basis announcement index: {i}")
                if (i == (self.num_nodes-node_num-1)):
                    j = 0
                    while j<(self.num_nodes-2):
#                         print(f"Node {node_num} send basis to node {list_classic[j+1][-1]}")
                        self.node.ports[list_classic[j+1]].tx_output(c)
                        j = j+1
#                 print(f"Node {node_num} wait basis from Node {list_classic[-1-i][-1]}")
                yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                 print(f"Node {node_num} Received basis from node {list_classic[-1-i][-1]}")
#                 print(message)
                basis_sum = basis_sum + message
                i= i+1
             #Send  basis
#             self.node.ports[list_classic[0]].tx_output(c)
#             self.node.ports[list_classic[1]].tx_output(c)
            
            #Record basis
            if (basis_sum % (self.num_nodes-1) == 0):
                if c == 0:
                    global_list[x][node_num-1] = measure_program1.output["M0"][0]
                else:
                     global_list[x][node_num-1] = measure_program2.output["M0"][0]
                basis_sum = 0
                x = x+1
                if (x > self.list_length-1):
                    if node_num == 3:
#                         print(f"List distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
#             print(f"Node {node_num} send ACK")
            self.node.ports[list_classic[0]].tx_output("ACK")
#             #Send measurement results
#             yield self.await_port_input(self.node.ports[list_classic[0]])
#             message = self.node.ports[list_classic[0]].rx_input().items[0]
#             print(message)
                
class RecvMeas_B1(NodeProtocol):
    def __init__(self, node ,name,prob, num_nodes,list_length):
        super().__init__(node, name)
        self.num_nodes = num_nodes
        self.list_length = list_length
        self.prob = prob
    def run(self):
#         print(f"Simulation start at {ns.sim_time(ns.MILLISECOND)} ms")
#         print(self.num_nodes)
#         qubit_number = int(np.log2(self.num_nodes))# Qubit number is log2 of number of nodes
#Init phase

        qubit_number = 1
        #Program to initialize the qubits in the memory, input param: number of qubits
        measure_program1 = MeasureZ(num_qubits=qubit_number)
        measure_program2 = MeasureX(num_qubits=qubit_number)
        randU_program = RandUnitary(num_qubits=qubit_number)
        #Variable to store classical and quantum ports
        list_port = [k for k in self.node.ports.keys()]
        list_classic = []
        list_quantum = []
        
        #Put classical ports in list_classic and quantum ports in list_quantum
#         print(list_port)
        for i in range(len(list_port)):
            if (list_port[i][0] == 'c'):
                list_classic.append(list_port[i])
            else:
                list_quantum.append(list_port[i])

        node_num = int(self.node.name.replace('P','')) # Current Node Number    
#         print(list_classic)
#         print(list_quantum)
        #Initialize basis count
        basis_sum = 0
        #Initialize loop count for number of state that has been distributed
        k = 0
        
        #Indicator variable for case of valid state (00) measurement
        valid_state = False
        
        #Initialize count for list length
        x = 0
        
# Program Start    
        #Loop For Program
        while True:
            #Wait for qubit
            yield self.await_port_input(self.node.ports[list_quantum[0]])
#             print(f"Node {node_num} received qubit")
#             print(self.node.qmemory.peek(0))
#             yield self.node.qmemory.execute_program(randU_program)
            #Measure qubit
            yield self.node.qmemory.execute_program(randU_program,prob=self.prob)
            c = random.randint(0,1)
#             c = 0
#             print(c)
            if c == 0:
#                 print(f"Node {node_num} measure in Z basis ")
                yield self.node.qmemory.execute_program(measure_program1,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program1.output)
#                 self.node.qmemory.discard(0)
            else:
#                 print(f"Node {node_num} measure in X basis ")
                yield self.node.qmemory.execute_program(measure_program2,mem_pos=[0])
#                 print(f"Node {node_num} output")
#                 print(measure_program2.output)
#                 self.node.qmemory.discard(0)
            basis_sum = c

            i=0
            while (i<self.num_nodes-2):
#                 print(f"Node {node_num} Loop for basis announcement index: {i}")
                if (i == (self.num_nodes-node_num-1)):
                    j = 0
                    while j<(self.num_nodes-2):
#                         print(f"Node {node_num} send basis to node {list_classic[j+1][-1]}")
                        self.node.ports[list_classic[j+1]].tx_output(c)
                        j = j+1
#                 print(f"Node {node_num} wait basis from Node {list_classic[-1-i][-1]}")
                yield self.await_port_input(self.node.ports[list_classic[-1-i]])
                message = self.node.ports[list_classic[-1-i]].rx_input().items[0]
#                 print(f"Node {node_num} Received basis from node {list_classic[-1-i][-1]}")
#                 print(message)
                basis_sum = basis_sum + message
                i= i+1
             #Send  basis
#             self.node.ports[list_classic[0]].tx_output(c)
#             self.node.ports[list_classic[1]].tx_output(c)
            
            #Record basis
            if (basis_sum % (self.num_nodes-1) == 0):
                if c == 0:
                    global_list[x][node_num-1] = measure_program1.output["M0"][0]
                else:
                     global_list[x][node_num-1] = measure_program2.output["M0"][0]
                basis_sum = 0
                x = x+1
                if (x > self.list_length-1):
                    if node_num == 3:
#                         print(f"List distribution ended at: {ns.sim_time(ns.MILLISECOND )} ms")
                        ns.sim_stop()
#             print(f"Node {node_num} send ACK")
            self.node.ports[list_classic[0]].tx_output("ACK")
#             #Send measurement results
#             yield self.await_port_input(self.node.ports[list_classic[0]])
#             message = self.node.ports[list_classic[0]].rx_input().items[0]
#             print(message)                

In [6]:
from netsquid.nodes import Network
def example_network_setup(num_nodes,prob,node_distance=4e-3):
#     print("Network Setup")
    nodes =[]
    i = 0
    while i<(num_nodes):
        if i ==1:
#             nodes.append(Node(f"P{i}",qmemory = create_processor1(prob)))
            nodes.append(Node(f"P{i}",qmemory = create_processor(prob)))
        else:
            nodes.append(Node(f"P{i}",qmemory = create_processor(prob)))
        
        i= i+1
    
    # Create a network
    network = Network("List Distribution Network")
#     print(nodes)
    network.add_nodes(nodes)
#     print("Nodes completed")
#     i = 0
#     while i< (num_nodes):
#         node = nodes[i]
#         j = 1
#         while j<=(num_nodes-i):
#             node_next = nodes[i+j]
#             c_conn = ClassicalBiConnection(name =f"c_conn{i}{i+j}", length = node_distance)
#             network.add_connection(node,node_next, connection= c_conn, label="classical", 
#                                    port_name_node1 = f"cio_node_port{i}{i+j}", port_name_node2 = f"cio_node_port{i+j}{i}")
#             j = j+1
#         i = i+1
    prob = 0
    c_conn = ClassicalBiConnection(name =f"c_conn{0}{1}", length = node_distance)
    network.add_connection(nodes[0],nodes[1], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{0}{1}", port_name_node2 = f"cio_node_port{1}{0}")
    c_conn = ClassicalBiConnection(name =f"c_conn{0}{2}", length = node_distance)
    network.add_connection(nodes[0],nodes[2], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{0}{2}", port_name_node2 = f"cio_node_port{2}{0}")
    c_conn = ClassicalBiConnection(name =f"c_conn{0}{3}", length = node_distance)
    network.add_connection(nodes[0],nodes[3], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{0}{3}", port_name_node2 = f"cio_node_port{3}{0}")
    c_conn = ClassicalBiConnection(name =f"c_conn{1}{2}", length = node_distance)
    network.add_connection(nodes[1],nodes[2], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{1}{2}", port_name_node2 = f"cio_node_port{2}{1}")
    c_conn = ClassicalBiConnection(name =f"c_conn{1}{3}", length = node_distance)
    network.add_connection(nodes[1],nodes[3], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{1}{3}", port_name_node2 = f"cio_node_port{3}{1}")
    c_conn = ClassicalBiConnection(name =f"c_conn{2}{3}", length = node_distance)
    network.add_connection(nodes[2],nodes[3], connection= c_conn, label="classical", 
                                   port_name_node1 = f"cio_node_port{2}{3}", port_name_node2 = f"cio_node_port{3}{2}")
    prob = 0
#     print("Classical Conn Completed")
    q_conn = QuantumConnection(name=f"qconn_{0}{1}", length=node_distance,prob=prob)
    network.add_connection(nodes[0], nodes[1], connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{0}{1}", port_name_node2=f"qin_node_port{1}{0}")
    q_conn = QuantumConnection(name=f"qconn_{0}{2}", length=node_distance,prob=prob)
    network.add_connection(nodes[0], nodes[2], connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{0}{2}", port_name_node2=f"qin_node_port{2}{0}")
    q_conn = QuantumConnection(name=f"qconn_{0}{3}", length=node_distance,prob=prob)
    network.add_connection(nodes[0], nodes[3], connection=q_conn, label="quantum", port_name_node1 = f"qo_node_port{0}{3}", port_name_node2=f"qin_node_port{3}{0}")

    i = 1
    while i<=(num_nodes-1):
        nodes[i].ports[f"qin_node_port{i}{0}"].forward_input(nodes[i].qmemory.ports['qin'])
        i = i+1
#     print("End Network Setup")
    
    return network

In [7]:
def setup_protocol(network,nodes_num,prob,fault_num,list_length):
#     print("Setup Protocol")
    
    protocol = LocalProtocol(nodes=network.nodes)
    nodes = []
    i = 0
    while i<(nodes_num):
        nodes.append(network.get_node("P"+str(i)))
        i = i+1
#     print(nodes)

    
    subprotocol = SourceInit(node=nodes[0],name=f"Source_init{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
#     subprotocol = FaultyInitSend(node=nodes[0],name=f"Faulty Init_Send{nodes[0].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    
    #Uncomment first line for normal phase reference, uncomment second line for phase reference error
#     subprotocol = RecvMeas(node=nodes[1], name=f"Receive_Measure{nodes[1].name}",num_nodes=nodes_num,list_length=list_length)
    subprotocol = RecvMeas_A(node=nodes[1], name=f"Receive_Measure{nodes[1].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    
#     subprotocol = RecvMeas_B(node=nodes[2], name=f"Receive_Measure{nodes[2].name}",num_nodes=nodes_num,list_length=list_length)
#     protocol.add_subprotocol(subprotocol)
    
    subprotocol = RecvMeas_B1(node=nodes[2], name=f"Receive_Measure{nodes[2].name}",num_nodes=nodes_num,prob=prob,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    
    subprotocol = RecvMeas_B(node=nodes[3], name=f"Receive_Measure{nodes[3].name}",num_nodes=nodes_num,list_length=list_length)
    protocol.add_subprotocol(subprotocol)
    return protocol


In [8]:
from netsquid.util.simtools import set_random_state
import pandas as pd

# set up initial parameters
nodes_num = 4 #Node numbers

fault_num = 0 #Faulty node numbers
exp_number = 1 #Experiment numbers
# probs = np.linspace(0, 1, num=100)


probs = np.linspace(0, 2*np.pi, num=100)


# print(probs)
exp_number = len(probs)
list_length = 100 #List length
error_array = np.ndarray(shape=(exp_number,2))
# time_array = np.ndarray(shape=(exp_number,))
global_list = np.ndarray(shape=(list_length,nodes_num-1), dtype='i')
x=0
# ns.sim_reset()
# network = example_network_setup(nodes_num,0,node_distance=4)
# protocol = setup_protocol(network,nodes_num,fault_num,list_length)
# protocol.start()
# stats = ns.sim_run()
# protocol = setup_protocol(network,nodes_num,fault_num,list_length)
average =100
while x < len(probs):
    basis = np.ndarray(shape=(list_length,nodes_num), dtype='i')
#     valid_basis = np.ndarray(shape=(list_length,nodes_num), dtype='i')
    global_list = np.ndarray(shape=(list_length,nodes_num-1), dtype='i')
    error_sum = 0
    for z in range(average):
        ns.sim_reset()
        network = example_network_setup(nodes_num,probs[x],node_distance=4)
#         protocol = setup_protocol(network,nodes_num,fault_num,list_length)
        protocol = setup_protocol(network,nodes_num,probs[x],fault_num,list_length)
        protocol.start()
        stats = ns.sim_run()

        if (ns.sim_state() == 2):
    #         print(f"Sim end time: {ns.sim_time()}")
    #         time_array[x] = ns.sim_time(ns.MILLISECOND)
    #         row_sum = 0
            valid_sum = 0
            for i in range(global_list.shape[0]-1):
                row_sum = 0
                for j in range (global_list.shape[1]):
                    row_sum = row_sum + global_list[i][j]
                if ((row_sum % (nodes_num-1)) == 0):
                    valid_sum = valid_sum+1
    #         percentage_correct = (valid_sum/(global_list.shape[0]-1)) * 100
            percentage_correct = (valid_sum/(global_list.shape[0]-1))
            error_ratio = 1-percentage_correct
            error_sum = error_sum + error_ratio
    print(f"Error Prob = {probs[x]/np.pi} pi Averaged Ratio of Error List:{error_sum/average}")
#         print(f"Percentage of Correct List: {round(percentage_correct,3)}%")
    error_array[x][0] = probs[x]
    error_array[x][1] = error_sum/average
    x = x+1
        
#     print(f"Error Prob = {probs[x]} Average Ratio of Error List:{error_sum/40}")
#     error_array[x][0] = probs[x]
#     error_array[x][1] = error_ratio/40
#     x = x+1



Error Prob = 0.0 pi Averaged Ratio of Error List:0.0
Error Prob = 0.020202020202020204 pi Averaged Ratio of Error List:0.0006060606060606033
Error Prob = 0.04040404040404041 pi Averaged Ratio of Error List:0.0014141414141414087
Error Prob = 0.060606060606060615 pi Averaged Ratio of Error List:0.005454545454545443
Error Prob = 0.08080808080808081 pi Averaged Ratio of Error List:0.006969696969696951
Error Prob = 0.10101010101010101 pi Averaged Ratio of Error List:0.011313131313131292
Error Prob = 0.12121212121212123 pi Averaged Ratio of Error List:0.016868686868686856
Error Prob = 0.14141414141414144 pi Averaged Ratio of Error List:0.024848484848484856
Error Prob = 0.16161616161616163 pi Averaged Ratio of Error List:0.03303030303030304
Error Prob = 0.18181818181818182 pi Averaged Ratio of Error List:0.03828282828282831
Error Prob = 0.20202020202020202 pi Averaged Ratio of Error List:0.04838383838383838
Error Prob = 0.2222222222222222 pi Averaged Ratio of Error List:0.0613131313131313
Err

Error Prob = 2.0 pi Averaged Ratio of Error List:0.0


In [9]:
# print(global_list)
# print(error_array)
error_data = pd.DataFrame(data = error_array,columns = ['error probability','Error List Ratio'])
print(error_data)
error_data.to_csv('Entangled_Phase_Error.csv')

    error probability  Error List Ratio
0            0.000000          0.000000
1            0.063467          0.000606
2            0.126933          0.001414
3            0.190400          0.005455
4            0.253866          0.006970
..                ...               ...
95           6.029319          0.008586
96           6.092786          0.004747
97           6.156252          0.002020
98           6.219719          0.000505
99           6.283185          0.000000

[100 rows x 2 columns]
